In [3]:
import logging
import numpy as np

logging.basicConfig(level=logging.DEBUG, format='%(asctime)s. %(levelname)s in %(funcName)s: %(message)s')

In [13]:
pwd

u'D:\\FF\\paco'

In [17]:
import paco.pacoUtils as pu

In [48]:
def readVectFF(f):
    """Reads a vector in the form output by FreeFem++.
    
    First line contains the number of entries: nbcoef. 
    Next lines contain the entries, with 5 entries per line, 
    separated by tab.
    Entries are read left to right, then line by line.
    """
    
    nbcoef = int(f.next().strip())
    logging.info("attempt to read %s values", nbcoef)
    arr = np.loadtxt(f)
    arr = arr.flatten(order='C')
    if len(arr) != nbcoef:
        errMsg = "readVectFF: Inconsistent dimension, expected %d, got %d" %(nbcoef, len(arr))
        logging.error(errMsg)
        raise RuntimeError(errMsg)
    logging.info("successfully read %s values", nbcoef)
    return arr
        

In [49]:
with open('..\generate\u.txt', 'r') as f:
    x = readVectFF(f)

In [50]:
x

array([ 0.02493221,  0.00731244,  0.05315321, ...,  0.41820378,
        0.42036873,  0.41557122])

In [51]:
len(x)

1370

In [55]:
def readSparseFF(f):
    """Reads a sparse matrix in the form (Morse) output by FreeFem++.
    
    First line contains: n m (is symmetic) nbcoef 
    Next lines contain, for each nonzero coefficient:   i j a_ij where (i,j) \in  {1,...,n}x{1,...,m} 
    """
    
    # strip comment header
    row = f.next().strip()
    while row[0] == '#':
        row = f.next().strip()
    # read shape
    n, m, issym, nbcoef = map(int, row.split())
    logging.info("sparse (%s x %s), sym? %s, attempt to read %s values", n, m, issym, nbcoef)
    # read nzeros
    ijv = np.loadtxt(f)
    nrows, ncols = ijv.shape
    # check dims
    if nrows != nbcoef:
        errMsg = "Inconsistent dimensions, expected %d, got %d" %(nbcoef, nrows)
        logging.error(errMsg)
        raise RuntimeError(errMsg)
    if ncols != 3:
        errMsg = "Inconsistent dimensions, expected %d, got %d" %(3, ncols)
        logging.error(errMsg)
        raise RuntimeError(errMsg)
    # populate matrix
    res = np.zeros((n,m))
    for ifloat,jfloat,v in ijv:
        i, j = int(ifloat) - 1, int(jfloat) - 1  # python is 0-based
        res[i,j] = v
        if issym != 0:
            res[j,i] = v

    logging.info("successfully read %s values", nbcoef)
    return res
        

In [56]:
with open('..\generate\stiff.txt', 'r') as f:
    s = readSparseFF(f)

In [57]:
s.shape

(736L, 736L)

In [1]:
pwd()

u'D:\\FF\\paco\\src'

In [19]:
Aref = pu.read_csr_matrix(open("runs\\bay\\bayREF_s2_A.txt", "r"))
Arun = pu.read_csr_matrix(open("runs\\bay\\bayrun_s2_A.txt", "r"))

2016-11-15 17:42:18,894. DEBUG in read_csr_matrix: attempting to read matrix in CSR format...
2016-11-15 17:42:18,894. DEBUG in read_csr_matrix: attempting to read (736 x 736) matrix in CSR format...
2016-11-15 17:42:18,911. INFO in read_csr_matrix: successfully read (736 x 736) matrix in CSR format with nnz 4946.
2016-11-15 17:42:18,937. DEBUG in read_csr_matrix: attempting to read matrix in CSR format...
2016-11-15 17:42:18,938. DEBUG in read_csr_matrix: attempting to read (736 x 736) matrix in CSR format...
2016-11-15 17:42:18,954. INFO in read_csr_matrix: successfully read (736 x 736) matrix in CSR format with nnz 4946.


In [27]:
Aref[0:6,0:6]

array([[ 18.24302629,  -2.12542865,  -4.74834635,   0.        ,
          0.        ,   0.        ],
       [ -4.95297701,  19.75467806,  -6.73354761,   0.        ,
          0.        ,  -1.12166326],
       [ -5.64640651,  -3.04448637,  27.49759487,  -3.64660761,
          0.        ,   0.        ],
       [  0.        ,   0.        ,  -3.13161298,  36.63096494,
         -4.6626828 ,   0.        ],
       [  0.        ,   0.        ,   0.        ,  -3.56103884,
         54.99933169,   0.        ],
       [  0.        ,  -5.53770732,   0.        ,   0.        ,
          0.        ,  21.31237229]])

In [28]:
Arun[0:6,0:6]

array([[ 14.99750919,  -2.12542865,  -4.74834635,   0.        ,
          0.        ,   0.        ],
       [ -3.53920283,  15.17993917,  -4.88901699,   0.        ,
          0.        ,  -1.12166326],
       [ -5.19737643,  -3.04448637,  19.55309687,  -3.3891103 ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,  -3.13161298,  24.75774094,
         -4.11186082,   0.        ],
       [  0.        ,   0.        ,   0.        ,  -3.56103884,
         35.23019745,   0.        ],
       [  0.        ,  -3.32968529,   0.        ,   0.        ,
          0.        ,  15.23584746]])

In [24]:
stiffref = pu.readSparseFF(open("runs\\bay\\bayREF_s1_stiff.txt", "r"))
stiffrun = pu.readSparseFF(open("runs\\bay\\bayrun_s1_stiff.txt", "r"))


2016-11-15 18:33:20,260. DEBUG in readSparseFF: attempt to read FF sparse matrix...
2016-11-15 18:33:20,263. DEBUG in readSparseFF: read sparse FF (736 x 736), sym? 0, 4946 values...
2016-11-15 18:33:20,313. INFO in readSparseFF: successfully read 4946 values from FF sparse matrix.
2016-11-15 18:33:20,315. DEBUG in readSparseFF: attempt to read FF sparse matrix...
2016-11-15 18:33:20,315. DEBUG in readSparseFF: read sparse FF (736 x 736), sym? 0, 4946 values...
2016-11-15 18:33:20,367. INFO in readSparseFF: successfully read 4946 values from FF sparse matrix.


In [25]:
np.array_equal(stiffref, stiffrun)

True

In [29]:
Aref[0:8,0:8]-Arun[0:8,0:8]

array([[  3.24551710e+00,   0.00000000e+00,  -8.88178420e-16,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00],
       [ -1.41377418e+00,   4.57473889e+00,  -1.84453062e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00],
       [ -4.49030081e-01,   0.00000000e+00,   7.94449800e+00,
         -2.57497312e-01,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,  -4.44089210e-16,
          1.18732240e+01,  -5.50821981e-01,   0.00000000e+00,
          0.00000000e+00,  -2.22044605e-16],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         -4.44089210e-16,   1.97691342e+01,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,  -2.20802203e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   6.07652483e+00,
         -2.18197888e+00,   0.0

In [32]:
np.isclose(Aref[0:8,0:8],Arun[0:8,0:8])

array([[False,  True,  True,  True,  True,  True,  True,  True],
       [False, False, False,  True,  True,  True,  True,  True],
       [False,  True, False, False,  True,  True,  True,  True],
       [ True,  True,  True, False, False,  True,  True,  True],
       [ True,  True,  True,  True, False,  True,  True,  True],
       [ True, False,  True,  True,  True, False, False,  True],
       [ True, False, False, False,  True,  True, False, False],
       [ True,  True,  True, False, False,  True,  True, False]], dtype=bool)

In [38]:
massref = pu.readSparseFF(open("runs\\bay\\bayREF_s1_mass.txt", "r"))
massrun = pu.readSparseFF(open("runs\\bay\\bayrun_s1_mass.txt", "r"))
massgen = pu.readSparseFF(open("generate\\mass.txt", "r"))
print(np.allclose(massref, massrun))
print(np.allclose(massref, massgen))
print(np.allclose(massrun, massgen))

2016-11-15 20:12:06,736. DEBUG in readSparseFF: attempt to read FF sparse matrix...
2016-11-15 20:12:06,737. DEBUG in readSparseFF: read sparse FF (736 x 736), sym? 0, 4946 values...
2016-11-15 20:12:06,798. INFO in readSparseFF: successfully read 4946 values from FF sparse matrix.
2016-11-15 20:12:06,799. DEBUG in readSparseFF: attempt to read FF sparse matrix...
2016-11-15 20:12:06,799. DEBUG in readSparseFF: read sparse FF (736 x 736), sym? 0, 4946 values...
2016-11-15 20:12:06,848. INFO in readSparseFF: successfully read 4946 values from FF sparse matrix.
2016-11-15 20:12:06,851. DEBUG in readSparseFF: attempt to read FF sparse matrix...
2016-11-15 20:12:06,851. DEBUG in readSparseFF: read sparse FF (736 x 736), sym? 0, 4946 values...
2016-11-15 20:12:06,905. INFO in readSparseFF: successfully read 4946 values from FF sparse matrix.


True
True
True


In [39]:
stiffref = pu.readSparseFF(open("runs\\bay\\bayREF_s1_stiff.txt", "r"))
stiffrun = pu.readSparseFF(open("runs\\bay\\bayrun_s1_stiff.txt", "r"))
stiffgen = pu.readSparseFF(open("generate\\stiff.txt", "r"))
print(np.allclose(stiffref, stiffrun))
print(np.allclose(stiffref, stiffgen))
print(np.allclose(stiffrun, stiffgen))

2016-11-15 20:13:00,687. DEBUG in readSparseFF: attempt to read FF sparse matrix...
2016-11-15 20:13:00,688. DEBUG in readSparseFF: read sparse FF (736 x 736), sym? 0, 4946 values...
2016-11-15 20:13:00,752. INFO in readSparseFF: successfully read 4946 values from FF sparse matrix.
2016-11-15 20:13:00,753. DEBUG in readSparseFF: attempt to read FF sparse matrix...
2016-11-15 20:13:00,753. DEBUG in readSparseFF: read sparse FF (736 x 736), sym? 0, 4946 values...
2016-11-15 20:13:00,809. INFO in readSparseFF: successfully read 4946 values from FF sparse matrix.
2016-11-15 20:13:00,809. DEBUG in readSparseFF: attempt to read FF sparse matrix...
2016-11-15 20:13:00,811. DEBUG in readSparseFF: read sparse FF (736 x 736), sym? 0, 4946 values...
2016-11-15 20:13:00,868. INFO in readSparseFF: successfully read 4946 values from FF sparse matrix.


True
True
True


In [41]:
uref = pu.readVectFF(open("runs\\bay\\bayREF_s1_u.txt", "r"))
urun = pu.readVectFF(open("runs\\bay\\bayrun_s1_u.txt", "r"))
ugen = pu.readVectFF(open("generate\\u.txt", "r"))
print(np.allclose(uref, urun))
print(np.allclose(uref, ugen))
print(np.allclose(urun, ugen))

2016-11-15 20:13:56,785. DEBUG in readVectFF: attempt to read FF vect...
2016-11-15 20:13:56,786. DEBUG in readVectFF: attempt to read 1370 values from FF vect
2016-11-15 20:13:56,789. INFO in readVectFF: successfully read 1370 values from FF vect.
2016-11-15 20:13:56,789. DEBUG in readVectFF: attempt to read FF vect...
2016-11-15 20:13:56,790. DEBUG in readVectFF: attempt to read 1370 values from FF vect
2016-11-15 20:13:56,793. INFO in readVectFF: successfully read 1370 values from FF vect.
2016-11-15 20:13:56,844. DEBUG in readVectFF: attempt to read FF vect...
2016-11-15 20:13:56,845. DEBUG in readVectFF: attempt to read 1370 values from FF vect
2016-11-15 20:13:56,846. INFO in readVectFF: successfully read 1370 values from FF vect.


True
True
True


In [42]:
vref = pu.readVectFF(open("runs\\bay\\bayREF_s1_v.txt", "r"))
vrun = pu.readVectFF(open("runs\\bay\\bayrun_s1_v.txt", "r"))
vgen = pu.readVectFF(open("generate\\v.txt", "r"))
print(np.allclose(vref, vrun))
print(np.allclose(vref, vgen))
print(np.allclose(vrun, vgen))

2016-11-15 20:14:48,625. DEBUG in readVectFF: attempt to read FF vect...
2016-11-15 20:14:48,627. DEBUG in readVectFF: attempt to read 1370 values from FF vect
2016-11-15 20:14:48,631. INFO in readVectFF: successfully read 1370 values from FF vect.
2016-11-15 20:14:48,632. DEBUG in readVectFF: attempt to read FF vect...
2016-11-15 20:14:48,634. DEBUG in readVectFF: attempt to read 1370 values from FF vect
2016-11-15 20:14:48,638. INFO in readVectFF: successfully read 1370 values from FF vect.
2016-11-15 20:14:48,647. DEBUG in readVectFF: attempt to read FF vect...
2016-11-15 20:14:48,648. DEBUG in readVectFF: attempt to read 1370 values from FF vect
2016-11-15 20:14:48,650. INFO in readVectFF: successfully read 1370 values from FF vect.


True
True
True
